頭２万データを使ってそれを16000と4000に分けるというやりかた。

In [1]:
import os
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn import decomposition
from sklearn import preprocessing
from sklearn import manifold
from sklearn import metrics
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import urllib.request
import shutil
import zipfile
from scipy.stats import mode

In [2]:
fold = "../train_raw_npy/"
xyz = np.load(f"{fold}acc_xyz.npy")
label = np.load(f"{fold}sampled_label.npy")
print(xyz.shape)
print(label.shape)


(196072, 3, 500)
(196072,)


In [3]:
xyz = xyz.reshape(xyz.shape[0], -1)
print(xyz.shape)

(196072, 1500)


In [4]:
from sklearn.preprocessing import StandardScaler

# データを2Dに変換して標準化
scaler = StandardScaler()
xyz_reshaped = xyz.reshape(-1, xyz.shape[-1])  # (196072*3, 500)
scaled_xyz = scaler.fit_transform(xyz_reshaped)  # 標準化
scaled_xyz = scaled_xyz.reshape(xyz.shape)  # 元の形に戻す

print(scaled_xyz.shape)  # (196072, 3, 500)


(196072, 1500)


0~20000を使う

In [5]:
xyz_20000 = scaled_xyz[-40000:-10000,:]
label_20000 = label[-40000:-10000]
print(xyz_20000.shape)
print(label_20000.shape)

(30000, 1500)
(30000,)


In [6]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(xyz_20000, label_20000, test_size=0.5, shuffle=True)

In [12]:
clf = BalancedRandomForestClassifier(
    n_estimators=2000,
    replacement=True,
    sampling_strategy='not minority',
    oob_score=True,
    n_jobs=6,
    random_state=42,
    verbose=1
)
clf.fit(x_train, y_train)

c:\Users\kaiha\anaconda3\envs\labo_Okita\lib\site-packages\imblearn\ensemble\_forest.py:601: FutureWarning: The default of `bootstrap` will change from `True` to `False` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `False` to silence this warning and adopt the future behaviour.
  warn(
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    2.6s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:   13.0s
[Parallel(n_jobs=6)]: Done 438 tasks      | elapsed:   30.0s
[Parallel(n_jobs=6)]: Done 788 tasks      | elapsed:   54.4s
[Parallel(n_jobs=6)]: Done 1238 tasks      | elapsed:  1.4min
[Parallel(n_jobs=6)]: Done 1788 tasks      | elapsed:  2.1min
[Parallel(n_jobs=6)]: Done 2000 out of 2000 | elapsed:  2.3min finished


BalancedRandomForestClassifier(n_estimators=2000, n_jobs=6, oob_score=True,
                               random_state=42, replacement=True,
                               sampling_strategy='not minority', verbose=1)

In [13]:
pred_label = clf.predict(x_test)

[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:    0.3s
[Parallel(n_jobs=6)]: Done 438 tasks      | elapsed:    0.7s
[Parallel(n_jobs=6)]: Done 788 tasks      | elapsed:    1.4s
[Parallel(n_jobs=6)]: Done 1238 tasks      | elapsed:    2.3s
[Parallel(n_jobs=6)]: Done 1788 tasks      | elapsed:    3.4s
[Parallel(n_jobs=6)]: Done 2000 out of 2000 | elapsed:    3.8s finished


In [14]:
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test, pred_label)

In [15]:
print(score)

0.6972666666666667


In [16]:
Y_test_pred = clf.predict(x_test) # 学習器の性能をみる テストデータで
print('\nClassifier performance')
print('Out of sample:\n', metrics.classification_report(y_test, Y_test_pred, zero_division=0))

[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:    0.3s
[Parallel(n_jobs=6)]: Done 438 tasks      | elapsed:    0.8s
[Parallel(n_jobs=6)]: Done 788 tasks      | elapsed:    1.5s
[Parallel(n_jobs=6)]: Done 1238 tasks      | elapsed:    2.5s
[Parallel(n_jobs=6)]: Done 1788 tasks      | elapsed:    3.6s



Classifier performance
Out of sample:
               precision    recall  f1-score   support

         1.0       0.68      0.61      0.64      2612
         2.0       0.78      0.73      0.76      1863
         3.0       0.98      0.98      0.98       601
         4.0       0.83      0.93      0.88      2445
         5.0       0.64      0.75      0.69      1868
         6.0       0.64      0.54      0.59      2260
         7.0       0.67      0.63      0.65      2443
         8.0       0.40      0.54      0.46       908

    accuracy                           0.70     15000
   macro avg       0.70      0.71      0.71     15000
weighted avg       0.70      0.70      0.70     15000



[Parallel(n_jobs=6)]: Done 2000 out of 2000 | elapsed:    4.1s finished


testを他のところでやってみる

In [17]:
test_x_addi = xyz[170000:180000, :]
test_y_addi = label[170000:180000]
print(test_x_addi.shape)
print(test_y_addi.shape)

(10000, 1500)
(10000,)


In [18]:
Y_test_pred_addi = clf.predict(test_x_addi) # 学習器の性能をみる テストデータで
print('\nClassifier performance')
print('Out of sample:\n', metrics.classification_report(test_y_addi, Y_test_pred_addi, zero_division=0))


Classifier performance
Out of sample:
               precision    recall  f1-score   support

         1.0       0.50      0.12      0.20      2148
         2.0       0.24      0.14      0.18       953
         3.0       0.84      0.95      0.89       503
         4.0       0.74      0.96      0.84      2157
         5.0       0.00      0.00      0.00      1929
         6.0       0.15      0.52      0.24       702
         7.0       0.08      0.04      0.05      1608
         8.0       0.00      0.00      0.00         0

    accuracy                           0.34     10000
   macro avg       0.32      0.34      0.30     10000
weighted avg       0.36      0.34      0.31     10000

